In [1]:
import torch
print("CUDA 지원 여부:", torch.cuda.is_available())  # 🔥 반드시 True여야 GPU 학습 가능
print("PyTorch 내부 CUDA 버전:", torch.version.cuda)
print("컴파일된 버전:", torch.__config__.show())  # 내부 컴파일 설정 보기

CUDA 지원 여부: True
PyTorch 내부 CUDA 버전: 12.1
컴파일된 버전: PyTorch built with:
  - C++ Version: 201703
  - MSVC 192930151
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.8.1  (built against CUDA 12.0)
  - Magma 2.5.4
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.8.1, CXX_COMPILER=C:/cb/pytorch_1000000000000/work/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR 

## 이진 분류 + 배치 학습

In [2]:
import pandas as pd
import numpy as np
import torch
df = pd.read_csv('data/breast_cancer.csv',index_col = 0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
df.head()

from sklearn.preprocessing import LabelEncoder

X = df.drop('target',axis=1).values

le = LabelEncoder()

y  = le.fit_transform(df['target'])

# y = torch.tensor()

print(X.shape,y.shape)

print(np.unique(y))



cuda
(569, 30) (569,)
[0 1]


In [3]:
X_tensor = torch.tensor(X,dtype=torch.float32)
y_tensor = torch.tensor(y,dtype=torch.float32).view(-1,1)

print(X_tensor.shape,y_tensor.shape)


torch.Size([569, 30]) torch.Size([569, 1])


In [4]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_tensor,y_tensor,test_size=0.2)

In [5]:
from torch.utils.data import DataLoader,TensorDataset

train_dataset = TensorDataset(X_train,y_train)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# y_test

In [6]:
import torch.nn as nn

class BinaryClassifier(nn.Module):

        def __init__(self,input_dim):

            super().__init__()
            self.linear1 = nn.Linear(input_dim,64)
            self.linear2 = nn.Linear(64,32)
            self.linear3 = nn.Linear(32,10)
            self.output = nn.Linear(10,1)
            self.relu = nn.ReLU()

        def forward(self,x):

            x = self.relu(self.linear1(x))
            x = self.relu(self.linear2(x))
            x = self.relu(self.linear3(x))
            x = (self.output(x))
            return torch.sigmoid(x)
        

model = BinaryClassifier(input_dim=X_train.shape[-1]).to(device)





In [7]:
# nn.BCELoss()
# nn.BCEWithLogitsLoss() -> sigmoid 가 포함됨 / sigmoid + BCELoss  

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [8]:
nb_epoch = 3000

for epoch in range(nb_epoch):

    model.train()
    total_loss = 0 

    for batch_x,batch_y in train_loader:
        batch_x,batch_y = batch_x.to(device) ,batch_y.to(device)
        pred = model(batch_x)
        loss = criterion(pred,batch_y)
        optimizer.zero_grad()

        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    
    if epoch % 100 == 0 :
        model.eval()
        with torch.no_grad():
            test_pred = model(X_test.to(device))
            # test_loss = criterion(test_pred,y_test)
            acc = ((test_pred >= 0.5).float() == y_test.to(device)).float().mean()
            print(f'{epoch}/ {nb_epoch} | test_acc {acc.item() * 100:.2f}')

0/ 3000 | test_acc 75.44
100/ 3000 | test_acc 93.86
200/ 3000 | test_acc 92.11
300/ 3000 | test_acc 87.72
400/ 3000 | test_acc 93.86
500/ 3000 | test_acc 95.61
600/ 3000 | test_acc 92.98
700/ 3000 | test_acc 95.61
800/ 3000 | test_acc 95.61
900/ 3000 | test_acc 95.61
1000/ 3000 | test_acc 92.98
1100/ 3000 | test_acc 92.98


KeyboardInterrupt: 

In [ ]:
# 정확도 처럼 활용 가능

((test_pred > 0.4).float() == y_test).float().mean()

tensor(0.9474)